Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [23]:
import numpy as np
import random as rnd
import lab9_lib

from tqdm import tqdm
from dataclasses import dataclass
from copy import deepcopy

## Numeric Values

In [9]:
PROBLEM_SIZE = 50
POPULATION_SIZE = 10
TOURNAMENT_SIZE = 3
OFFSPRINGS = 10
GENERATIONS = 5
MUTATION_PROBABILITY = .3

## Evolutionary Strategy Functions

In [21]:
@dataclass
class Individual:
    genotype: str
    fitness: float

    def __str__(self):
        return f"{self.genotype}: {self.fitness:.2%}"

    def fitness(self) -> float:
        return fitness

def mutation(ind: Individual) -> Individual:
    gene = rnd.choice([i in range(PROBLEM_SIZE)])
    offspring = deepcopy(ind)
    offspring.genotype[gene] = 1 - offspring.genotype[gene] 
    offspring.fitness = None
    return offspring

def one_cut_xover(ind1: Individual, ind2: Individual) -> Individual:
    cut_point = int(len(ind1.genotype) / 2)
    offspring = Individual(genotype=np.concatenate((ind1.genotype[:cut_point], ind2.genotype[cut_point:]), axis=None), fitness=None)
    return offspring

def select_parent(population: list) -> Individual:
    pool = [rnd.choice(population) for _ in range(TOURNAMENT_SIZE)]
    champion = max(pool, key=lambda i: i.fitness)
    return champion

## Population Generation

In [26]:
fitness = lab9_lib.make_problem(PROBLEM_SIZE)
# for n in range(PROBLEM_SIZE):
#     ind = choices([0, 1], k=50)
#     print(f"{''.join(str(g) for g in ind)}: {fitness(ind):.2%}")

population = [
    Individual(
        genotype = rnd.choices([0, 1], k=50),
        fitness = None,
    )
    for _ in range(POPULATION_SIZE)
]

for i in population:
    i.fitness = fitness(i.genotype)

## Training

In [27]:
for gen in tqdm(range(GENERATIONS)):
    print(f"Generation {gen}/{GENERATIONS}:")
    
    offsprings = list()
    for _ in range(OFFSPRINGS):
        p1 = select_parent(population)
        p2 = select_parent(population)
        o = one_cut_xover(p1, p2)

        if rnd.random() < MUTATION_PROBABILITY:
            o = mutation(o)
        
        offsprings.append(o)

    for o in offsprings:
        o.fitness = fitness(i.genotype)

    population.extend(offsprings)
    population.sort(key=lambda i: i.fitness, reverse=True)
    population = population[:POPULATION_SIZE]

    print(f"Best individual of the generation: {population[0]}")

print(f"Number of fitness calls: {fitness.calls}")

100%|██████████| 5/5 [00:00<?, ?it/s]

Generation 0/5:
Best individual of the generation: [0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0]: 58.00%
Generation 1/5:
Best individual of the generation: [0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0]: 58.00%
Generation 2/5:
Best individual of the generation: [0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0]: 58.00%
Generation 3/5:
Best individual of the generation: [0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0]: 58.00%
Generation 4/5:
Best individual of the generation: [0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 